_We're going to be using pandas an json, so lets start with that._

In [164]:
import json
import pandas as pd
import os
from pandas.io.json import json_normalize

# Looking at the json files

_Before we do any real work, can we list the files from our target directory?_

In [165]:
!pwd
! ls ../APIs

/home/vagrant/git/BigData/democratizing_weather_data/democratizing_weather_data/streaming/prototypes
2yahoo.json	     wsdot.TrafficFlows.json  yahoo_weather1line.json
API_index.csv	     wsdot.Traveltimes.json   yahoo_weather2.json
openweathermap.json  yahoo_weather	      yahoo_weather.json


_Nice! Let's examine a json file_

In [166]:
!head ../APIs/yahoo_weather.json

{
    "query": {
        "count": 1,
        "created": "2017-07-29T23:51:25Z",
        "lang": "en-US",
        "diagnostics": {
            "publiclyCallable": "true",
            "url": {
                "execution-start-time": "1",
                "execution-stop-time": "5",


# Exploring JSON from the Yahoo Weather API

_Read a json file from disk (we'll use ../API/<tbd>.json
We should reall use the yahoo api output, since we know how to convert that to a csv_

Source [Andy Hayden on ghthub.io](http://hayd.github.io/2013/pandas-json)

In [167]:
df = pd.read_json('../APIs/yahoo_weather.json')
df

,query
count,1
created,2017-07-29T23:51:25Z
diagnostics,"{'publiclyCallable': 'true', 'url': {'executio..."
lang,en-US
results,"{'channel': {'units': {'distance': 'mi', 'pres..."


_Not much of a dataframe. Let's re-read and nomralize, drilling into the results section._

Read file with json Library. Source: [Scott Robinson's Stack Abuse Article ](http://stackabuse.com/reading-and-writing-json-to-a-file-in-python/)

In [168]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_df = json_normalize(raw_json['query']) 
yahoo_weather_df

,count,created,diagnostics.build-version,diagnostics.publiclyCallable,diagnostics.service-time,diagnostics.url.content,diagnostics.url.execution-start-time,diagnostics.url.execution-stop-time,diagnostics.url.execution-time,diagnostics.user-time,...,results.channel.location.region,results.channel.title,results.channel.ttl,results.channel.units.distance,results.channel.units.pressure,results.channel.units.speed,results.channel.units.temperature,results.channel.wind.chill,results.channel.wind.direction,results.channel.wind.speed
0,1,2017-07-29T23:51:25Z,2.0.164,true,4,http://weather-ydn-yql.media.yahoo.com:4080/v3...,1,5,4,6,...,CA,"Yahoo! Weather - Sunnyvale, CA, US",60,mi,in,mph,F,75,355,18


_One row, 50 columns. Let's look at data types._

In [169]:
yahoo_weather_df.dtypes

count                                     int64
created                                  object
diagnostics.build-version                object
diagnostics.publiclyCallable             object
diagnostics.service-time                 object
diagnostics.url.content                  object
diagnostics.url.execution-start-time     object
diagnostics.url.execution-stop-time      object
diagnostics.url.execution-time           object
diagnostics.user-time                    object
lang                                     object
results.channel.astronomy.sunrise        object
results.channel.astronomy.sunset         object
results.channel.atmosphere.humidity      object
results.channel.atmosphere.pressure      object
results.channel.atmosphere.rising        object
results.channel.atmosphere.visibility    object
results.channel.description              object
results.channel.image.height             object
results.channel.image.link               object
results.channel.image.title             

_That's a lot of data, all of type object. For the Yahoo Weather API, most of the good stuff lives in_ results.channel
_Let's dig deeper_

In [170]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_df = json_normalize(raw_json['query']['results']['channel']) 
yahoo_weather_df

,astronomy.sunrise,astronomy.sunset,atmosphere.humidity,atmosphere.pressure,atmosphere.rising,atmosphere.visibility,description,image.height,image.link,image.title,...,location.region,title,ttl,units.distance,units.pressure,units.speed,units.temperature,wind.chill,wind.direction,wind.speed
0,6:10 am,8:19 pm,58,1006.0,0,16.1,"Yahoo! Weather for Sunnyvale, CA, US",18,http://weather.yahoo.com,Yahoo! Weather,...,CA,"Yahoo! Weather - Sunnyvale, CA, US",60,mi,in,mph,F,75,355,18


_Looking more like a dataframe.  Let's look at fields & datatypes_

In [171]:
yahoo_weather_df.dtypes

astronomy.sunrise        object
astronomy.sunset         object
atmosphere.humidity      object
atmosphere.pressure      object
atmosphere.rising        object
atmosphere.visibility    object
description              object
image.height             object
image.link               object
image.title              object
image.url                object
image.width              object
item.condition.code      object
item.condition.date      object
item.condition.temp      object
item.condition.text      object
item.description         object
item.forecast            object
item.guid.isPermaLink    object
item.lat                 object
item.link                object
item.long                object
item.pubDate             object
item.title               object
language                 object
lastBuildDate            object
link                     object
location.city            object
location.country         object
location.region          object
title                    object
ttl     

_Everything's an object.  If we want to do any real analysis, we're going to need to do some type conversion. Also note, that the field names have a hierarchy to them.  So maybe we can drill deeper into just the fields we want_

_Drill Down: atmosphere_

In [172]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_atmosphere_df = json_normalize(raw_json['query']['results']['channel']['atmosphere']) 
yahoo_weather_atmosphere_df

,humidity,pressure,rising,visibility
0,58,1006.0,0,16.1


_Drill Down: image_

In [173]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_image_df = json_normalize(raw_json['query']['results']['channel']['image']) 
yahoo_weather_image_df

,height,link,title,url,width
0,18,http://weather.yahoo.com,Yahoo! Weather,http://l.yimg.com/a/i/brand/purplelogo//uh/us/...,142


_Drill Down: item_

In [174]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_item_df = json_normalize(raw_json['query']['results']['channel']['item']) 
yahoo_weather_item_df

,condition.code,condition.date,condition.temp,condition.text,description,forecast,guid.isPermaLink,lat,link,long,pubDate,title
0,32,"Sat, 29 Jul 2017 04:00 PM PDT",75,Sunny,"<![CDATA[<img src=""http://l.yimg.com/a/i/us/we...","[{'code': '34', 'date': '29 Jul 2017', 'day': ...",false,37.371609,http://us.rd.yahoo.com/dailynews/rss/weather/C...,-122.038254,"Sat, 29 Jul 2017 04:00 PM PDT","Conditions for Sunnyvale, CA, US at 04:00 PM PDT"


_Drill Down: item.forecast_

In [175]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_forecast_df = json_normalize(raw_json['query']['results']['channel']['item']['forecast']) 
yahoo_weather_forecast_df

,code,date,day,high,low,text
0,34,29 Jul 2017,Sat,78,58,Mostly Sunny
1,32,30 Jul 2017,Sun,84,60,Sunny
2,32,31 Jul 2017,Mon,87,61,Sunny
3,32,01 Aug 2017,Tue,88,64,Sunny
4,34,02 Aug 2017,Wed,90,66,Mostly Sunny
5,30,03 Aug 2017,Thu,88,68,Partly Cloudy
6,34,04 Aug 2017,Fri,88,67,Mostly Sunny
7,32,05 Aug 2017,Sat,86,66,Sunny
8,34,06 Aug 2017,Sun,83,63,Mostly Sunny
9,34,07 Aug 2017,Mon,79,61,Mostly Sunny


_Drill Down:condition_

In [176]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_conditions_df = json_normalize(raw_json['query']['results']['channel']['item']['condition']) 
yahoo_weather_conditions_df

,code,date,temp,text
0,32,"Sat, 29 Jul 2017 04:00 PM PDT",75,Sunny


_Drill Down: wind_

In [177]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_atmosphere_df = json_normalize(raw_json['query']['results']['channel']['wind']) 
yahoo_weather_atmosphere_df

,chill,direction,speed
0,75,355,18


# Exploring JSON from the Open Weather Map API

In [178]:
!pwd
! ls ../APIs
! cat ../APIs/openweathermap.json

/home/vagrant/git/BigData/democratizing_weather_data/democratizing_weather_data/streaming/prototypes
2yahoo.json	     wsdot.TrafficFlows.json  yahoo_weather1line.json
API_index.csv	     wsdot.Traveltimes.json   yahoo_weather2.json
openweathermap.json  yahoo_weather	      yahoo_weather.json
{
    "coord": {
        "lon": 139.01,
        "lat": 35.02
    },
    "weather": [
        {
            "id": 800,
            "main": "Clear",
            "description": "clear sky",
            "icon": "01n"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 285.514,
        "pressure": 1013.75,
        "humidity": 100,
        "temp_min": 285.514,
        "temp_max": 285.514,
        "sea_level": 1023.22,
        "grnd_level": 1013.75
    },
    "wind": {
        "speed": 5.52,
        "deg": 311
    },
    "clouds": {
        "all": 0
    },
    "dt": 1485792967,
    "sys": {
        "message": 0.0025,
        "country": "JP",
        "sunrise": 1485726240,
        "sunset"

In [179]:
with open('../APIs/openweathermap.json') as json_file:
  raw_json = json.load(json_file)

openweathermap_df = json_normalize(raw_json) 
openweathermap_df

,base,clouds.all,cod,coord.lat,coord.lon,dt,id,main.grnd_level,main.humidity,main.pressure,...,main.temp_max,main.temp_min,name,sys.country,sys.message,sys.sunrise,sys.sunset,weather,wind.deg,wind.speed
0,stations,0,200,35.02,139.01,1485792967,1907296,1013.75,100,1013.75,...,285.514,285.514,Tawarano,JP,0.0025,1485726240,1485763863,"[{'id': 800, 'main': 'Clear', 'description': '...",311,5.52


In [180]:
openweathermap_df.dtypes

base                object
clouds.all           int64
cod                  int64
coord.lat          float64
coord.lon          float64
dt                   int64
id                   int64
main.grnd_level    float64
main.humidity        int64
main.pressure      float64
main.sea_level     float64
main.temp          float64
main.temp_max      float64
main.temp_min      float64
name                object
sys.country         object
sys.message        float64
sys.sunrise          int64
sys.sunset           int64
weather             object
wind.deg             int64
wind.speed         float64
dtype: object

_May of the data types are already numeric.  Pretty cool. Let's dive into the weather object._

In [181]:
with open('../APIs/openweathermap.json') as json_file:
  raw_json = json.load(json_file)

openweathermap_weather_df = json_normalize(raw_json['weather']) 
openweathermap_weather_df

,description,icon,id,main
0,clear sky,01n,800,Clear


# One folder, one dataframe from multiple json files

The yahoo\_weather sufolder contains two json files from the Yahoo weather API.  We've moved all the linefeeds, but that won't affect our ability to make dataframes

In [182]:
! ls -1 ../APIs/yahoo_weather

yahoo_weather1line.json
yahoo_weather2.json


_Uncomment the lines below to see the contets of the files_

In [183]:
#! head ../APIs/yahoo_weather/yahoo_weather1line.json

#! echo ""
#! echo "===== NEXT FILE ==="
#! echo ""

#! cat ../APIs/yahoo_weather/yahoo_weather2.json

In [184]:
top_folder = "../APIs/yahoo_weather"
first_time = True

for filename in os.listdir(top_folder):
    filepath = top_folder + "/" + filename

    with open(filepath) as json_file:
      raw_json = json.load(json_file)

    new_json = json_normalize(raw_json['query']['results']['channel']['atmosphere'])
    
    if (first_time):
        first_time = False
        multifile_df = new_json
    else:
        multifile_df = multifile_df.append(new_json, ignore_index=True)
    
multifile_df

,humidity,pressure,rising,visibility
0,44,1019.0,0,16.1
1,58,1006.0,0,16.1


In [185]:
multifile_df.dtypes

humidity      object
pressure      object
rising        object
visibility    object
dtype: object

# One file, multiple json objects

_File 2yahoo.json contains two json objects from the yahoo weather api.  Each object has had newlines removed, so it only occupies one line of text. Uncomment the line below to have a look._

In [186]:
# !cat ../APIs/2yahoo.json

_The linux word count (wc) with the line (-l) option can count the lines int he file)_

In [187]:
!wc -l < ../APIs/2yahoo.json

2


_Read the json file into a dataframe ... what do we get?_

In [188]:
df3 = pd.read_json('../APIs/2yahoo.json', lines=True)
df3

,query
0,"{'count': 1, 'created': '2017-07-29T23:51:25Z'..."
1,"{'count': 1, 'created': '2017-08-01T02:14:07Z'..."


_... not much.  Nice that we got a mulit-row dataframe, but this json needs to be normalized to be useful_

Code is pretty similar to the multi-file case, just using json.loads() instead of json.load()
Source: [Stack Overflow](https://stackoverflow.com/questions/8009882/how-to-read-large-file-line-by-line-in-python)

In [189]:
first_time = True

with open('../APIs/2yahoo.json') as f_one_json_per_line:
    for line in f_one_json_per_line:
        raw_json = json.loads(line)
        new_json = json_normalize(raw_json['query']['results']['channel']['atmosphere'])
        
        if (first_time):
            first_time = False
            multiline_df = new_json
        else:
            multiline_df = multiline_df.append(new_json, ignore_index=True)
        
multiline_df

,humidity,pressure,rising,visibility
0,58,1006.0,0,16.1
1,44,1019.0,0,16.1


# WSDOT Exploring Transportation Data

In [190]:
wsdot_TravelTimes_df = pd.read_json('../APIs/wsdot.Traveltimes.json')
wsdot_TravelTimes_df

,AverageTime,CurrentTime,Description,Distance,EndPoint,Name,StartPoint,TimeUpdated,TravelTimeID
0,30,33,Everett to Downtown Seattle using HOV lanes,26.72,{'Description': 'I-5 @ University St in Seattl...,HOV Everett-Seattle (SB AM),"{'Description': 'I-5 @ 41st St in Everett', 'D...",/Date(1500575100000-0700)/,2
1,30,29,Downtown Seattle to Everett using HOV lanes,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",HOV Seattle-Everett (NB PM),{'Description': 'I-5 @ University St in Seattl...,/Date(1500575100000-0700)/,3
2,30,29,Downtown Seattle to Everett,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett (NB PM),{'Description': 'I-5 @ University St in Seattl...,/Date(1500575100000-0700)/,4
3,11,11,Downtown Bellevue to Issaquah,9.55,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah (EB PM),{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1500575100000-0700)/,5
4,11,11,Downtown Bellevue to Issaquah using HOV lanes,9.55,"{'Description': 'I-90 @ Front St in Issaquah',...",HOV Bellevue-Issaquah (EB PM),{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1500575100000-0700)/,6
5,10,11,Issaquah to Downtown Bellevue using HOV lanes,9.48,{'Description': 'I-405 @ NE 8th St in Bellevue...,HOV Issaquah-Bellevue (WB AM),"{'Description': 'I-90 @ Front St in Issaquah',...",/Date(1500575100000-0700)/,7
6,12,12,Issaquah to Downtown Bellevue,9.48,{'Description': 'I-405 @ NE 8th St in Bellevue...,Issaquah-Bellevue (WB AM),"{'Description': 'I-90 @ Front St in Issaquah',...",/Date(1500575100000-0700)/,8
7,28,27,Downtown Bellevue to Everett,26.06,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Bellevue-Everett (NB PM),{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1500575100000-0700)/,9
8,30,30,Everett to Downtown Bellevue,26.34,{'Description': 'I-405 @ NE 8th St in Bellevue...,Everett-Bellevue (SB AM),"{'Description': 'I-5 @ 41st St in Everett', 'D...",/Date(1500575100000-0700)/,10
9,26,27,Downtown Bellevue to Everett using HOV lanes,26.06,"{'Description': 'I-5 @ 41st St in Everett', 'D...",HOV Bellevue-Everett (NB PM),{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1500575100000-0700)/,11


In [191]:
wsdot_TravelTimes_df.dtypes

AverageTime       int64
CurrentTime       int64
Description      object
Distance        float64
EndPoint         object
Name             object
StartPoint       object
TimeUpdated      object
TravelTimeID      int64
dtype: object

In [195]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_df = json_normalize(raw_json['query']) 
yahoo_weather_df

,count,created,diagnostics.build-version,diagnostics.publiclyCallable,diagnostics.service-time,diagnostics.url.content,diagnostics.url.execution-start-time,diagnostics.url.execution-stop-time,diagnostics.url.execution-time,diagnostics.user-time,...,results.channel.location.region,results.channel.title,results.channel.ttl,results.channel.units.distance,results.channel.units.pressure,results.channel.units.speed,results.channel.units.temperature,results.channel.wind.chill,results.channel.wind.direction,results.channel.wind.speed
0,1,2017-07-29T23:51:25Z,2.0.164,true,4,http://weather-ydn-yql.media.yahoo.com:4080/v3...,1,5,4,6,...,CA,"Yahoo! Weather - Sunnyvale, CA, US",60,mi,in,mph,F,75,355,18


In [198]:
with open('../APIs/wsdot.Traveltimes.json') as json_file:
  raw_json = json.load(json_file)

wsdot_TravelTimes_df = json_normalize(raw_json[0]['StartPoint'])
print(wsdot_TravelTimes_df)

                Description Direction  Latitude  Longitude  MilePost RoadName
0  I-5 @ 41st St in Everett         S  47.92428 -122.26548    192.55      I-5


In [199]:
wsdot_TravelTimes_df.dtypes

Description     object
Direction       object
Latitude       float64
Longitude      float64
MilePost       float64
RoadName        object
dtype: object

In [203]:
with open('../APIs/wsdot.Traveltimes.json') as json_file:
  file_data = json_file.read()
    
list_json = json.loads(file_data)

with open('../APIs/wsdot.Traveltimes.json') as json_file:
  raw_json = json.load(json_file)

ii = 0

travel_df= json_normalize(raw_json[ii]['StartPoint'])

for travelTime in list_json:
  wsdot_TravelTimes_df = json_normalize(raw_json[ii]['StartPoint'])
  #print(wsdot_TravelTimes_df)
  travel_df = travel_df.append(wsdot_TravelTimes_df, ignore_index=True)
  ii= ii+ 1
   
travel_df

,Description,Direction,Latitude,Longitude,MilePost,RoadName
0,I-5 @ 41st St in Everett,S,47.924280,-122.265480,192.55,I-5
1,I-5 @ 41st St in Everett,S,47.924280,-122.265480,192.55,I-5
2,I-5 @ University St in Seattle,N,47.609294,-122.331759,165.83,I-5
3,I-5 @ University St in Seattle,N,47.609294,-122.331759,165.83,I-5
4,I-405 @ NE 8th St in Bellevue,S,47.613800,-122.188920,13.33,I-405
5,I-405 @ NE 8th St in Bellevue,S,47.613800,-122.188920,13.33,I-405
6,I-90 @ Front St in Issaquah,W,47.541799,-122.037396,16.96,I-90
7,I-90 @ Front St in Issaquah,W,47.541799,-122.037396,16.96,I-90
8,I-405 @ NE 8th St in Bellevue,N,47.613610,-122.187970,13.60,I-405
9,I-5 @ 41st St in Everett,S,47.924280,-122.265480,192.55,I-5


In [204]:
travel_df.describe()

,Latitude,Longitude,MilePost
count,181.000000,181.000000,181.000000
mean,46.310329,-120.323464,89.062044
std,6.086943,15.665790,80.895078
min,0.000000,-122.891484,0.100000
25%,46.973870,-122.504230,13.330000
50%,47.464800,-122.331130,105.380000
75%,47.613610,-122.206552,165.830000
max,48.152323,0.000000,307.900000


In [205]:
with open('../APIs/wsdot.Traveltimes.json') as json_file:
  file_data = json_file.read()
    
list_json = json.loads(file_data)

with open('../APIs/wsdot.Traveltimes.json') as json_file:
  raw_json = json.load(json_file)

ii = 0

travel_df= json_normalize(raw_json[ii]['StartPoint'])

for travelTime in list_json:
  wsdot_TravelTimes_df = json_normalize(raw_json[ii]['EndPoint'])
  travel_df = travel_df.append(wsdot_TravelTimes_df, ignore_index=True)
  ii= ii+ 1

travel_df

,Description,Direction,Latitude,Longitude,MilePost,RoadName
0,I-5 @ 41st St in Everett,S,47.924280,-122.265480,192.55,I-5
1,I-5 @ University St in Seattle,S,47.609294,-122.331759,165.83,I-5
2,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,I-5
3,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,I-5
4,I-90 @ Front St in Issaquah,E,47.541799,-122.037396,16.96,I-90
5,I-90 @ Front St in Issaquah,E,47.541799,-122.037396,16.96,I-90
6,I-405 @ NE 8th St in Bellevue,N,47.613610,-122.187970,13.60,I-405
7,I-405 @ NE 8th St in Bellevue,N,47.613610,-122.187970,13.60,I-405
8,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,I-5
9,I-405 @ NE 8th St in Bellevue,S,47.613800,-122.188920,13.33,I-405


In [206]:
travel_df.describe()

,Latitude,Longitude,MilePost
count,181.000000,181.000000,181.000000
mean,47.077500,-122.365980,99.785083
std,0.859322,0.240875,89.247853
min,45.353550,-122.891484,0.110000
25%,47.048034,-122.560450,13.330000
50%,47.464800,-122.331759,121.040000
75%,47.613610,-122.219631,165.830000
max,48.152323,-120.599140,307.350000


# Future Work
- Figure out how to join and merge stuff (geogson)
